#데이터 수집 코드(우리는 필요 없음)

In [ ]:
!pip install icrawler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from icrawler.builtin import GoogleImageCrawler
google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4,
                                    storage={'root_dir': '/content/drive/MyDrive/장기_프로젝트/new_model'})

google_crawler.crawl(keyword='car crash', max_num=500,
                     date_min=None, date_max=None,
                     min_size=(200,200), max_size=None)

TypeError: ignored

In [ ]:
!gdown 1EcAeCLDTlHXf9jzBiSTbqIHMSF6WZC03

Downloading...
From: https://drive.google.com/uc?id=1EcAeCLDTlHXf9jzBiSTbqIHMSF6WZC03
To: /content/512_padding_data_220722.zip
100% 59.5M/59.5M [00:00<00:00, 255MB/s]


In [ ]:
!unzip /content/512_padding_data_220722.zip

#데이터 이미지 train, test 나눠주기\이미지를 rgb로 변환 후 64, 64크기로 리사이즈 되어있음(우리 데이터 크기로 조절하기)

In [ ]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split

# 분류 대상 카테고리 선택하기 
accident_dir = "/content/drive/MyDrive/장기_프로젝트/new_model/make_data/1"
categories = ['images', 'masks']
nb_classes = len(categories)
# 이미지 크기 지정 
image_w = 512
image_h = 512
pixels = image_w * image_h * 3
# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 이미지 
    image_dir = accident_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    for i, f in enumerate(files):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
X = np.array(X)
Y = np.array(Y)
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = \
    train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)

print('>>> data 저장중 ...')
# np.save("./image/7obj.npy", xy)
print("ok,", len(Y))

#모델 학습

hdf5 파일로 저장

In [ ]:
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

# 카테고리 지정하기
categories = ['images', 'masks']
nb_classes = len(categories)
# 이미지 크기 지정하기
image_w = 64
image_h = 64
# 데이터 열기 
X_train, X_test, y_train, y_test = np.load("./image/7obj.npy")
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)

# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 전결합층
model.add(Flatten())    # 벡터형태로 reshape
model.add(Dense(512))   # 출력
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
# 모델 구축하기
model.compile(loss='categorical_crossentropy',   # 최적화 함수 지정
    optimizer='rmsprop',
    metrics=['accuracy'])
# 모델 확인
#print(model.summary())

# 학습 완료된 모델 저장
hdf5_file = "./image/7obj-model.hdf5"
if os.path.exists(hdf5_file):
    # 기존에 학습된 모델 불러들이기
    model.load_weights(hdf5_file)
else:
    # 학습한 모델이 없으면 파일로 저장
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10)
    model.save_weights(hdf5_file)

#정확도 오차, 정확도 확인

In [ ]:
score = model.evaluate(X_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

#데이터 예측 확인

In [ ]:
test_image = './image/test_overturned.jpg'
# 이미지 resize
img = Image.open(test_image)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])

In [ ]:
!gdown 1weHJqe9aGPH1I6bP1K7FzRglIcFfW1fH

Downloading...
From: https://drive.google.com/uc?id=1weHJqe9aGPH1I6bP1K7FzRglIcFfW1fH
To: /content/bin_train_data.zip
100% 107M/107M [00:00<00:00, 362MB/s] 


In [ ]:
!gdown 17xTOcDDZPlmZTz4kn1oTozjt_mdulrNz

Downloading...
From: https://drive.google.com/uc?id=17xTOcDDZPlmZTz4kn1oTozjt_mdulrNz
To: /content/bin_valid_data.zip
100% 31.7M/31.7M [00:00<00:00, 131MB/s]


In [ ]:
!unzip /content/bin_train_data.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: aug_data/2/masks/mix_a_067.jpg  
  inflating: aug_data/2/masks/HorizontalFlip_a_020.jpg  
  inflating: aug_data/2/masks/HorizontalFlip_a_063.jpg  
  inflating: aug_data/2/masks/Transpose_b_0154.jpg  
  inflating: aug_data/2/masks/ShiftScaleRotate_a_094.jpg  
  inflating: aug_data/2/masks/ShiftScaleRotate_a_032.jpg  
  inflating: aug_data/2/masks/HorizontalFlip_b_0177.jpg  
  inflating: aug_data/2/masks/Blur_b_0162.jpg  
  inflating: aug_data/2/masks/VerticalFlip_b_0179.jpg  
  inflating: aug_data/2/masks/mix_b_0161.jpg  
  inflating: aug_data/2/masks/HorizontalFlip_a_069.jpg  
  inflating: aug_data/2/masks/a_06.jpg  
  inflating: aug_data/2/masks/RandomRotate90_b_0182.jpg  
  inflating: aug_data/2/masks/Blur_b_0131.jpg  
  inflating: aug_data/2/masks/mix_b_0164.jpg  
  inflating: aug_data/2/masks/a_083.jpg  
  inflating: aug_data/2/masks/Blur_a_035.jpg  
  inflating: aug_data/2/masks/b_0188.jpg  
  inflating: aug_data/2/masks/mix_b_0176.

In [ ]:
!unzip /content/bin_valid_data.zip

In [ ]:
!git clone https://github.com/luyanger1799/Amazing-Semantic-Segmentation.git

Cloning into 'Amazing-Semantic-Segmentation'...
remote: Enumerating objects: 1936, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 1936 (delta 189), reused 182 (delta 182), pack-reused 1696
Receiving objects: 100% (1936/1936), 572.47 MiB | 28.85 MiB/s, done.
Resolving deltas: 100% (353/353), done.
Checking out files: 100% (1472/1472), done.


In [ ]:
# !pip install numpy
# !pip install pillow
# !pip install opencv-python
!pip install tensorflow===1.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 70 kB/s 
     |████████████████████████████████| 3.1 MB 76.8 MB/s 
     |████████████████████████████████| 488 kB 85.3 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.1
    Uninstalling tensorflow-2.9.1:
      Successfully uninstalled tensorflow-2.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [ ]:
# !pip uninstall tensorflow

!pip install tensorflow-gpu===1.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 377.1 MB 9.0 kB/s 
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.9.1
    Uninstalling tensorflow-gpu-2.9.1:
      Successfully uninstalled tensorflow-gpu-2.9.1


In [ ]:
cd /content/Amazing-Semantic-Segmentation

/content/Amazing-Semantic-Segmentation


In [ ]:
!pwd

/content/Amazing-Semantic-Segmentation


In [ ]:
!python train.py --model FCN-8s --base_model ResNet50 --dataset /content/Amazing-Semantic-Segmentation/CamVid --num_classes 32

# 다른 모델

In [5]:
!gdown 1SSSgxzhcKj-Jptt424KBajSJhH6kvSs-

Downloading...
From: https://drive.google.com/uc?id=1SSSgxzhcKj-Jptt424KBajSJhH6kvSs-
To: /content/unet-nested-multiple-classification/DeepLabv3Plus_Multi_PNGdata.zip
100% 31.4M/31.4M [00:00<00:00, 209MB/s]


In [ ]:
!unzip /content/unet-nested-multiple-classification/DeepLabv3Plus_Multi_PNGdata.zip

In [7]:
!git clone https://github.com/zonasw/unet-nested-multiple-classification.git

Cloning into 'unet-nested-multiple-classification'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 92 (delta 31), reused 48 (delta 7), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [8]:
%cd /content/unet-nested-multiple-classification

/content/unet-nested-multiple-classification


In [ ]:
!python train.py

#Deeplabv3

In [1]:
!git clone https://github.com/lizhengwei1992/mobilenetv2_deeplabv3_pytorch.git

Cloning into 'mobilenetv2_deeplabv3_pytorch'...
remote: Enumerating objects: 77, done.
remote: Total 77 (delta 0), reused 0 (delta 0), pack-reused 77
Unpacking objects: 100% (77/77), done.


In [1]:
!gdown 1SSSgxzhcKj-Jptt424KBajSJhH6kvSs-

Downloading...
From: https://drive.google.com/uc?id=1SSSgxzhcKj-Jptt424KBajSJhH6kvSs-
To: /content/DeepLabv3Plus_Multi_PNGdata.zip
100% 31.4M/31.4M [00:00<00:00, 57.1MB/s]


In [ ]:
!unzip /content/DeepLabv3Plus_Multi_PNGdata.zip

In [5]:
a = cv2.imread('/content/content/Dataset/label1/train/Images/a_010.jpg')
a.shape

(512, 512, 3)

In [9]:
%cd /content/deeplabv3

/content/deeplabv3


In [11]:
!python train.py

Traceback (most recent call last):
  File "train.py", line 8, in <module>
    from deeplabv3 import DeepLabV3
ModuleNotFoundError: No module named 'deeplabv3'


#2번

In [1]:
# !git clone https://github.com/lizhengwei1992/mobilenetv2_deeplabv3_pytorch.git
!git clone https://github.com/luyanger1799/Amazing-Semantic-Segmentation.git

Cloning into 'Amazing-Semantic-Segmentation'...
remote: Enumerating objects: 1936, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1936 (delta 189), reused 183 (delta 183), pack-reused 1696
Receiving objects: 100% (1936/1936), 572.47 MiB | 49.05 MiB/s, done.
Resolving deltas: 100% (353/353), done.
Checking out files: 100% (1472/1472), done.


In [2]:
%cd /content/Amazing-Semantic-Segmentation

/content/Amazing-Semantic-Segmentation


In [3]:
!python train.py

Traceback (most recent call last):
  File "train.py", line 9, in <module>
    from utils.data_generator import ImageDataGenerator
  File "/content/Amazing-Semantic-Segmentation/utils/data_generator.py", line 9, in <module>
    from tensorflow.python.keras.preprocessing.image import Iterator
ModuleNotFoundError: No module named 'tensorflow.python.keras.preprocessing'


In [5]:
!pip install numpy
!pip install pillow
!pip install opencv-python
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.5 kB/s 
     |████████████████████████████████| 438 kB 68.4 MB/s 
     |████████████████████████████████| 1.6 MB 71.5 MB/s 
     |████████████████████████████████| 5.8 MB 59.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboa

In [ ]:
python train.py --model FCN-8s --base_model ResNet50 --dataset "dataset_path" --num_classes "num_classes"